In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# %load_ext autoreload
# %autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.kalman_filters import get_spread

In [4]:
DATA_PATH = r"data"
BACKTEST =  DATA_PATH + r"\backtest.h5"
UNIVERSE = DATA_PATH + r"\etfs_stocks_universe.h5"
ASSETS = DATA_PATH + r"\assets.h5"

In [5]:
with pd.HDFStore(BACKTEST) as store:
    candidates = store['candidates']
    prices = store['prices']

In [6]:
with pd.HDFStore(UNIVERSE) as store:
    etfs = store['etfs']
    stocks = store['stocks']

In [19]:
candidates

,test_end,eg,y,x
25,2017-12-31,0.009509,UNH.US,SPY.US
28,2017-12-31,0.048496,VZ.US,SPY.US
36,2017-12-31,0.009472,SPY.US,ADBE.US
45,2017-12-31,0.018519,LLY.US,SPY.US
67,2017-12-31,0.006577,SPY.US,AXP.US
...,...,...,...,...
24268,2023-12-31,0.044375,FXD.US,EQIX.US
24270,2023-12-31,0.006768,FXD.US,RCL.US
24273,2023-12-31,0.001703,ADSK.US,FXD.US
24275,2023-12-31,0.037516,FXD.US,FI.US


In [8]:
# candidates_sub = candidates[(candidates['test_end'] >= '2022-12-31') & (candidates['test_end'] < '2023-12-31')].sort_index()
# candidates_sub = candidates
# candidates_sub = candidates_sub.sort_values(['test_end','eg']).groupby(['test_end']).head(10)
# top_pairs = candidates_sub.drop(['eg'], axis = 1)
top_pairs = candidates[(candidates['test_end'] == '2017-12-31') & ((candidates['x'] == 'SPY.US') | (candidates['y'] == 'SPY.US'))].sort_values(['eg']).drop('eg', axis=1).head(5)

In [9]:
pairs, half_lives = get_spread(top_pairs, pd.concat([etfs, stocks], axis = 1))
pairs_df = pd.concat(pairs)

In [14]:
def get_trades(data):
    pair_trades = []
    for i, ((period, s1, s2), pair) in enumerate(data.groupby(['period','s1','s2']), 1):
        # first3m = pair.first('3M').index
        # last3m = pair.last('3M').index
        first1Y = pair.first('5Y').index

        entry = ((pair['z_score'].abs() > 2) & (np.sign(pair['z_score'].shift().fillna(method = 'bfill')) == np.sign(pair['z_score'])))
        entry = (entry.shift() != entry) * np.sign(pair['z_score']).fillna(0).astype(int) - 2
        exit = (np.sign(pair['z_score'].shift().fillna(method = 'bfill')) != np.sign(pair['z_score'])).astype(int) - 1

        trades = pd.concat((entry[entry != -2], exit[exit == 0])).to_frame('side').sort_values(['date','side']).squeeze()
        if not isinstance(trades, pd.Series):
            continue
        
        try:
            trades.loc[trades < 0] += 2
        except:
            break
        
        trades = trades[trades.abs().shift() != trades.abs()]
        window = trades.loc[first1Y.min():first1Y.max()]
        # extra = trades.loc[last3m.min():last3m.max()]
        n = len(trades)

        if window.iloc[0] == 0:
            if n > 1:
                print('shift')
                window = window.iloc[1:]
        if window.iloc[-1] != 0:
            continue
            # extra_exits = extra[extra == 0].head(1)
            # if extra_exits.empty:
            #     continue
            # else:
            #     window = pd.concat((window, extra_exits))

        trades = pair[['s1', 's2', 'hedge_ratio', 'period', 'pair']].join(window.to_frame('side'), how = 'right')
        trades.loc[trades.side == 0, 'hedge_ratio'] = np.nan
        trades.hedge_ratio = trades.hedge_ratio.ffill()
        pair_trades.append(trades)
    return pd.concat(pair_trades)

In [15]:
pair_trades = get_trades(pairs_df)

In [16]:
pair_trades

,s1,s2,hedge_ratio,period,pair,side
date,,,,,,
2018-01-02,D.US,SPY.US,-0.265354,1,1,-1
2018-01-08,D.US,SPY.US,-0.265354,1,1,0
2018-01-02,DD.US,SPY.US,-0.371571,1,3,1
2018-01-08,DD.US,SPY.US,-0.371571,1,3,0
2018-01-02,GME.US,SPY.US,-0.015997,1,4,1
...,...,...,...,...,...,...
2022-10-10,SPY.US,MPC.US,-3.808096,1,2,0
2022-11-22,SPY.US,MPC.US,-3.130123,1,2,-1
2022-11-28,SPY.US,MPC.US,-3.130123,1,2,0


In [18]:
pair_trades.to_hdf('data/backtest.h5','test_spy_top5-2/trades')